In [1]:
#  package import
import warnings   #------------------------------------------------------
warnings.filterwarnings("ignore")

import os
from os.path import join
import missingno as msno

import pandas as pd
import numpy as np
import missingno as msno

from sklearn.model_selection import KFold, cross_val_score  
from sklearn.model_selection import train_test_split        
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor 
from lightgbm import LGBMRegressor 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
#1) data read
data_dir = os.getenv('HOME')+'/aiffel/98_aiff_report/220921_Datathon'

aisles_data_path = join(data_dir, 'aisles.csv')
departments_data_path = join(data_dir, 'departments.csv')
order_products__prior_data_path = join(data_dir, 'order_products__prior.csv')
order_products__train_data_path = join(data_dir, 'order_products__train.csv')
orders_data_path = join(data_dir, 'orders.csv')
products_data_path = join(data_dir, 'products.csv')

aisles = pd.read_csv(aisles_data_path)
departments = pd.read_csv(departments_data_path)
order_products__prior = pd.read_csv(order_products__prior_data_path)
order_products__train = pd.read_csv(order_products__train_data_path )
orders = pd.read_csv(orders_data_path)
products = pd.read_csv(products_data_path)

In [17]:
# merging products and aisles df based on product id
temp1 = pd.merge(left=products, right=aisles, how='left')
# merging products and departments df based on product id
product_aisles_departments = pd.merge(left=temp1, right=departments, how='left')

product_aisles_departments.head()

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,2,All-Seasons Salt,104,13,spices seasonings,pantry
2,3,Robust Golden Unsweetened Oolong Tea,94,7,tea,beverages
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,frozen meals,frozen
4,5,Green Chile Anytime Sauce,5,13,marinades meat preparation,pantry


In [19]:
order_products__prior.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
 #   Column             Dtype
---  ------             -----
 0   order_id           int64
 1   product_id         int64
 2   add_to_cart_order  int64
 3   reordered          int64
dtypes: int64(4)
memory usage: 989.8 MB


In [ ]:
# merging order prior and temp1 based on prior
temp2 = pd.merge(left=order_products__prior, right=product_aisles_departments, how='left')
# merging order prior and orders df based on prior
#opp_order_prod_aisles_dept = pd.merge(left=temp2, right=orders, how='left')
temp2.head()

In [7]:
order_products__prior.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
 #   Column             Dtype
---  ------             -----
 0   order_id           int64
 1   product_id         int64
 2   add_to_cart_order  int64
 3   reordered          int64
dtypes: int64(4)
memory usage: 989.8 MB


In [11]:
order_products__prior['add_to_cart_order'].unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145])

In [3]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
dtypes: float64(1), int64(5), object(1)
memory usage: 182.7+ MB


In [6]:
#고객 : 206209
len(orders['user_id'].unique())

206209

In [ ]:
RFM_data = orders.groupby(['user_id']).agg({'days_since_prior_order':'min','order_id': 'count', 'Payment': 'sum'})
# RFM_data의 열 값을 각각 Recency, Frequency, Monetary로 변경
RFM_data.columns = ['Recency', 'Frequency', 'Monetary']
RFM_data.head()
